# CTE (computation expressions)

Ovaj notebook objašnjava sve CTE koje smo koristili na posljednjim vježbama. Konkretno: `promise` (Fable.Fetch/Fable.Promise) i `async` (F# Async).

## Kratka napomena o podacima u browseru

U browser-u nemamo direktan pristup lokalnom file sistemu, pa se podaci i fajlovi preuzimaju preko mrežnih protokola.
Zbog toga se koristi `fetch` i `promise` workflow za asinhrono dohvatanje sadržaja.


## Korišteni CTEs:

- `promise { ... }` radi nad Javascript Promise vrijednostima (Fable).
- `async { ... }` je vezan za F# Async workflow.


## CTE 1: `promise` iz Fable.Promise

`promise { ... }` je Fable computation expression koji omogućava da vežemo simbole za vrijednosti unutar Javascript promise-a.


In [ ]:
let task () =
  promise {
    let! r = Fetch.fetch "/words_alpha.txt" []
    let! text = r.text()
    return text.Split('\n') |> Set.ofArray
  }

**Kako `promise` CTE radi**

- `let!` sačeka Javascript promise i veže simbol za resolved vrijednost.
- `return` postavlja vrijednost cijelog `promise` bloka.
- Ako se baci iznimka ili se vrati rejected promise, cijeli blok se odbija (rejected).
- Izraz iznad rezultuje sa tipom: `Promise<Set<string>>`

**Interakcija sa Elmish Cmd**

- `Cmd<'msg>` u Elmishu je opis side efekata koje runtime izvršava nakon `update`, kako bi `update` ostao čist.
- `Cmd.OfPromise.perform` pretvara funkciju `unit -> Promise<'a>` u komandu; Elmish je poziva i, kad se promise resolve, `dispatch`-a poruku iz `('a -> 'msg)`.
- Ako želite eksplicitno upravljati greškama, koristiti `Cmd.OfPromise.either` ili `Cmd.OfPromise.attempt` koji mapiraju i grešku u poruku.

Primjeri potpisa (skraćeno):

`Cmd.OfPromise.perform : (unit -> Promise<'a>) -> unit -> ('a -> 'msg) -> Cmd<'msg>`

`Cmd.OfPromise.either  : (unit -> Promise<'a>) -> unit -> ('a -> 'msg) -> (exn -> 'msg) -> Cmd<'msg>`

Dokumentacija:
- Fable.Fetch: https://fable.io/fable-promise/
- Fable.Promise (promise CTE): https://fable.io/fable-promise/documentation/computation-expression.html
- F# computation expressions (opšti pregled): https://learn.microsoft.com/en-us/dotnet/fsharp/language-reference/computation-expressions


**Primjer: `promise` + `Cmd.OfPromise.perform`**

Ovo je minimalni primjer kako se `promise` workflow pretvara u Elmish komandu koja dispatch-a poruku kada se promise resolve.


In [ ]:
type Msg =
  | DataLoaded of string

let loadData () =
  promise {
    let! r = Fetch.fetch "/api/data" []
    return! r.text()
  }

let cmd : Cmd<Msg> =
  Cmd.OfPromise.perform loadData () DataLoaded


**Detalji za `Cmd.OfPromise.perform`**

`perform` uzima 3 argumenta (u ovom redoslijedu):

1. Funkciju tipa `'a -> Promise<'b>` koja pokrece asinhroni posao. Funkcija uzima bilo koji tip za prvi argument (u našem primjeru unit, vraća nazad, drugi, bilo koji tip koji je spakovan u promise. Kada se promise resolve-a moći ćemo pristupiti ovom drugom tipu).
2. Argument tipa `'a` koji je isti tip kao argument naše funkcije iz prethodnog koraka koji se proslijedjuje toj istoj funkciji.
3. Funkciju tipa `'b -> 'msg` koja mapira resolve-ani rezultat (tip koji je unutar konteksta promise-a) u poruku za `update`.

Elmish runtime pozove (1) sa argumentom (2), sačeka da se promise resolve-a na vrijednost tipa `'b`, pa zatim pozove (3) i dispatch-a dobijenu poruku.


## CTE 2: `async` (F# Async workflow)

U grani kada je selekcija slova neispravna, koristili smo `async` za kratko čekanje prije poruke `StopShaking`:


In [ ]:
let task () =
  async {
    do! Async.Sleep 300
  }

**Sta se desava:**

- `async { ... }` je F# computation expression za asinhroni workflow.
- `do! Async.Sleep 300` čeka 300 ms bez blokiranja.
- Elmish poziva `Cmd.OfAsync.perform` da nakon završetka pošalje `StopShaking`.

**Veza `async` sa Elmish Cmd (analogno `promise`)**

- `Cmd.OfAsync.perform` radi isto sto i `Cmd.OfPromise.perform`, ali očekuje F# `Async<'b>` umjesto JS `Promise<'b>`.
- Tipovi su analogni: funkcija pokreće `Async<'b>`, argument se proslijedi toj funkciji, a mapper `('b -> 'msg)` pretvara rezultat u poruku.

Skraćeni potpis:

`Cmd.OfAsync.perform : ('a -> Async<'b>) -> 'a -> ('b -> 'msg) -> Cmd<'msg>`


## Razlika između Promise-a i Async-a
Promise i Async tipovi se koriste za asinhrone operacije - operacije koje će se odraditi u budućnosti. 

- `promise` radi nad JS promise vrijednostima (Fable), koristi se za Fetch i interop sa JS. Obično se koristi za dohvatanje udaljenih podataka koji zahtjevaju transfer preko mreže. Samim time ne znamo koliko će trajati prenos podataka koji ovisi od količine podataka i same mreže ispod.
- `async` je F# workflow, koristan za kontrolu vremena u .NET stilu.
- U Elmish programu oba se pretvaraju u komande (`Cmd.OfPromise.perform` i `Cmd.OfAsync.perform`).